# Gesture UI Project 1 2025

***

## Dataset Option 1: Human Activity Recognition Using Smartphones

In [ ]:
# imports
import os
import numpy as np
import seaborn as sns
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import matplotlib.pyplot as plt


### Define File Paths and Load Feature Names

In this step, we define the file paths for the dataset, targets, and features files:
- `dataset_file`: Path to the dataset file (`dataset.txt`).
- `targets_file`: Path to the target labels file (`targets.txt`).
- `features_file`: Path to the feature names file (`features.txt`).

We then load the feature names from `features.txt`:
- The file is read line by line.
- Each line is stripped of leading/trailing whitespace and split by spaces.
- The second element (index `1`) of each line is extracted as the feature name.

In [ ]:
# file paths
dataset_file = "dataset.txt"
targets_file = "targets.txt"
features_file = "features.txt"

# Load feature names
with open(features_file, "r") as f:
    feature_names = [line.strip().split(" ")[1] for line in f.readlines()]


print("Labels loaded successfully!")


### Load the Dataset

In this step, we load the dataset from the file specified by `dataset_file`. The dataset is read using `pandas.read_csv` with the following parameters:
- `delim_whitespace=True`: Indicates that the file is space-separated.
- `header=None`: Specifies that the file does not have a header row.
- `names=unique_feature_names`: Assigns the unique feature names (previously loaded) as column names.

In [23]:
df_features = pd.read_csv(dataset_file, delim_whitespace=True, header=None, names=unique_feature_names)

print("Dataset loaded successfully!")
print(f"Dataset shape: {df_features.shape}")  # Verify correct shape


### Ensure Unique Feature Names

In this step, we ensure that all feature names in the dataset are unique. This is important because duplicate feature names can cause issues during data processing and modeling.

1. **Count Feature Occurrences**:
   - We use `Counter` from the `collections` module to count how many times each feature name appears in the list of feature names.

2. **Generate Unique Names**:
   - For features that appear more than once, we append a suffix (e.g., `_1`, `_2`, etc.) to make them unique.
   - Features that appear only once are kept as is.

3. **Assign Unique Names to the Dataset**:
   - The updated unique feature names are assigned as the column names of the dataset (`df_features`).

In [24]:
# Ensure unique feature names
feature_counts = Counter(feature_names)
unique_feature_names = []
feature_seen = {}

for feature in feature_names:
    if feature_counts[feature] > 1:
        feature_seen[feature] = feature_seen.get(feature, 0) + 1
        unique_feature_names.append(f"{feature}_{feature_seen[feature]}")
    else:
        unique_feature_names.append(feature)

# Assign new unique names to the dataset
df_features.columns = unique_feature_names

print("Duplicate feature names resolved!")

### Load Activity Labels and Merge with Features

In this step, we load the activity labels from the `targets.txt` file and merge them with the feature dataset (`df_features`) to create a complete dataset.

1. **Load Activity Labels**:
   - The activity labels are loaded from `targets.txt` using `pandas.read_csv`.
   - The file is space-separated (`delim_whitespace=True`), and the column is named `"Activity"`.

2. **Merge Features and Labels**:
   - The feature dataset (`df_features`) and the activity labels (`df_labels`) are merged along the columns (`axis=1`) to create a single DataFrame (`df`).

3. **Preview the Dataset**:
   - We print the first few rows of the merged dataset to verify that the merge was successful.

In [31]:
# Load activity labels
df_labels = pd.read_csv(targets_file, delim_whitespace=True, header=None, names=["Activity"])

# Merge features and labels
df = pd.concat([df_features, df_labels], axis=1)

print("Dataset and labels merged successfully!")
print(df.head())  # Preview dataset


***

### Define Features and Target Labels, and Split the Dataset

In this step, we prepare the dataset for training and testing by defining the features (`X`) and target labels (`y`), and then splitting the data into training and testing sets.

1. **Define Features (`X`) and Target Labels (`y`)**:
   - `X` contains all the feature columns, excluding the `"Activity"` column.
   - `y` contains the target labels, which are the values from the `"Activity"` column.

2. **Split the Dataset**:
   - The dataset is split into training and testing sets using an 80-20 split:
     - 80% of the data is used for training (`X_train`, `y_train`).
     - 20% of the data is used for testing (`X_test`, `y_test`).
   - The `stratify=y` parameter ensures that the class distribution is preserved in both the training and testing sets.
   - The `random_state=42` parameter ensures reproducibility of the split.

3. **Print Dataset Shapes**:
   - The shapes of the training and testing sets are printed to verify the split.

This step prepares the data for model training and evaluation.

In [26]:
# Define features (X) and target labels (y)
X = df.drop(columns=["Activity"])  # Feature columns
y = df["Activity"]  # Target labels

# Split into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training Set: {X_train.shape}, Test Set: {X_test.shape}")


### Check Class Distribution in Training and Test Sets

In this step, we verify the distribution of classes in both the training and testing sets to ensure that the dataset split has preserved the class proportions.

1. **Training Set Distribution**:
   - We use `value_counts()` to count the number of instances for each class in the training set (`y_train`).
   - The class distribution is printed to confirm that the training set has a balanced representation of all classes.

2. **Test Set Distribution**:
   - Similarly, we use `value_counts()` to count the number of instances for each class in the test set (`y_test`).

This step ensures that the dataset split (using `stratify=y`) has successfully maintained the same class distribution in both the training and testing sets, which is crucial for fair model evaluation.

In [27]:
# Check class distribution in training set
train_distribution = y_train.value_counts()
print("Training Set Distribution:\n", train_distribution)

# Check class distribution in test set
test_distribution = y_test.value_counts()
print("Test Set Distribution:\n", test_distribution)


### Standardize Features Using StandardScaler

In this step, we standardize the feature data to ensure that all features have a mean of 0 and a standard deviation of 1. This is important because many machine learning algorithms perform better when the input features are on the same scale.

1. **Initialize the Scaler**:
   - We use `StandardScaler` from `sklearn.preprocessing` to standardize the features.

2. **Fit and Transform the Training Data**:
   - The scaler is fitted to the training data (`X_train`) to compute the mean and standard deviation.
   - The training data is then transformed using the computed mean and standard deviation.

3. **Transform the Test Data**:
   - The test data (`X_test`) is transformed using the same mean and standard deviation computed from the training data. This ensures that the test data is scaled consistently with the training data.


Standardizing the features helps improve the performance and convergence of many machine learning models, especially those that are sensitive to the scale of the input data (e.g., SVM, logistic regression).

In [28]:
# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the training data, then transform the test data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Features standardized successfully!")


### Verify Scaled Data and Labels

In this step, we verify the scaled training data and the corresponding labels to ensure that the standardization process has been applied correctly.

1. **Check Scaled Training Data**:
   - We print the first 5 rows of the scaled training data (`X_train_scaled`) to confirm that the features have been standardized (mean = 0, standard deviation = 1).

2. **Check Corresponding Labels**:
   - We print the first 5 labels (`y_train`) to verify that they match the scaled training data.

This step ensures that the data preprocessing steps (e.g., standardization) have been applied correctly and that the data is ready for model training.

In [29]:
print(X_train_scaled[:5])  # Check first 5 rows of scaled training data
print(y_train[:5])  # Check first 5 labels


***

### Train a Logistic Regression Model

In this step, we initialize and train a Logistic Regression model using the standardized training data.

1. **Initialize the Model**:
   - We use `LogisticRegression` from `sklearn.linear_model`.
   - The `max_iter=2000` parameter ensures that the model converges by allowing up to 2000 iterations.
   - The `random_state=42` parameter ensures reproducibility of the results.

2. **Train the Model**:
   - The model is trained on the scaled training data (`X_train_scaled`) and the corresponding labels (`y_train`).

Logistic Regression is a commonly used classification algorithm that works well for binary and multi-class classification tasks. Standardizing the features ensures that the model performs optimally.

In [32]:
# Initialize the Logistic Regression model
lr_model = LogisticRegression(max_iter=2000, random_state=42)

# Train the model on the training set
lr_model.fit(X_train_scaled, y_train)

print("Logistic Regression model trained successfully!")

### Make Predictions Using the Logistic Regression Model

In this step, we use the trained Logistic Regression model to make predictions on the scaled test data.

1. **Generate Predictions**:
   - The `predict` method is used to generate predictions (`y_pred_lr`) for the test data (`X_test_scaled`).

These predictions will be used to evaluate the performance of the Logistic Regression model by comparing them to the actual labels (`y_test`).

In [33]:
# Predict on test data
y_pred_lr = lr_model.predict(X_test_scaled)

print("Predictions made successfully!")


### Evaluate the Logistic Regression Model

In this step, we evaluate the performance of the Logistic Regression model using various evaluation metrics.

1. **Compute Evaluation Metrics**:
   - **Accuracy**: Measures the proportion of correctly classified instances.
   - **Precision**: Measures the proportion of true positive predictions among all positive predictions (weighted average is used for multi-class classification).
   - **Recall**: Measures the proportion of true positives identified correctly (weighted average is used for multi-class classification).
   - **F1-Score**: The harmonic mean of precision and recall, providing a balance between the two (weighted average is used for multi-class classification).

2. **Print the Classification Report**:
   - The `classification_report` function provides a detailed breakdown of precision, recall, F1-score, and support for each class.

3. **Print Evaluation Scores**:
   - The accuracy, precision, recall, and F1-score are printed to summarize the model's performance.

These metrics help us understand how well the Logistic Regression model is performing on the test data and identify areas for improvement.

In [42]:
# Compute evaluation metrics
accuracy_lr = accuracy_score(y_test, y_pred_lr)
precision_lr = precision_score(y_test, y_pred_lr, average="weighted")
recall_lr = recall_score(y_test, y_pred_lr, average="weighted")
f1_lr = f1_score(y_test, y_pred_lr, average="weighted")

# Print the classification report
print("Logistic Regression Classification Report:\n")
print(classification_report(y_test, y_pred_lr))

# Print evaluation scores
print(f"Accuracy: {accuracy_lr:.4f}")
print(f"Precision: {precision_lr:.4f}")
print(f"Recall: {recall_lr:.4f}")
print(f"F1-Score: {f1_lr:.4f}")


***

### Train a Random Forest Model

In this step, we initialize and train a Random Forest model using the standardized training data.

1. **Initialize the Model**:
   - We use `RandomForestClassifier` from `sklearn.ensemble`.
   - The `n_estimators=100` parameter specifies the number of trees in the forest.
   - The `random_state=42` parameter ensures reproducibility of the results.

2. **Train the Model**:
   - The model is trained on the scaled training data (`X_train_scaled`) and the corresponding labels (`y_train`).

Random Forest is an ensemble learning method that combines multiple decision trees to improve accuracy and reduce overfitting. It is robust and works well for both classification and regression tasks.

In [35]:
# Initialize the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model on the training set
rf_model.fit(X_train_scaled, y_train)

print("Random Forest model trained successfully!")


### Make Predictions Using the Random Forest Model

In this step, we use the trained Random Forest model to make predictions on the scaled test data.

1. **Generate Predictions**:
   - The `predict` method is used to generate predictions (`y_pred_rf`) for the test data (`X_test_scaled`).

These predictions will be used to evaluate the performance of the Random Forest model by comparing them to the actual labels (`y_test`).

In [36]:
# Predict on test data
y_pred_rf = rf_model.predict(X_test_scaled)

print("Predictions made successfully!")


### Evaluate the Random Forest Model

In this step, we evaluate the performance of the Random Forest model using various evaluation metrics.

1. **Compute Evaluation Metrics**:
   - **Accuracy**: Measures the proportion of correctly classified instances.
   - **Precision**: Measures the proportion of true positive predictions among all positive predictions (weighted average is used for multi-class classification).
   - **Recall**: Measures the proportion of true positives identified correctly (weighted average is used for multi-class classification).
   - **F1-Score**: The harmonic mean of precision and recall, providing a balance between the two (weighted average is used for multi-class classification).

2. **Print the Classification Report**:
   - The `classification_report` function provides a detailed breakdown of precision, recall, F1-score, and support for each class.

3. **Print Evaluation Scores**:
   - The accuracy, precision, recall, and F1-score are printed to summarize the model's performance.

These metrics help us understand how well the Random Forest model is performing on the test data and identify areas for improvement.

In [43]:
# Compute evaluation metrics
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf, average="weighted")
recall_rf = recall_score(y_test, y_pred_rf, average="weighted")
f1_rf = f1_score(y_test, y_pred_rf, average="weighted")

# Print the classification report
print("Random Forest Classification Report:\n")
print(classification_report(y_test, y_pred_rf))

# Print evaluation scores
print(f"Accuracy: {accuracy_rf:.4f}")
print(f"Precision: {precision_rf:.4f}")
print(f"Recall: {recall_rf:.4f}")
print(f"F1-Score: {f1_rf:.4f}")

***

### Train a Support Vector Machine (SVM) Model

In this step, we initialize and train a Support Vector Machine (SVM) model using the standardized training data.

1. **Initialize the Model**:
   - We use `SVC` (Support Vector Classification) from `sklearn.svm`.
   - The `kernel="linear"` parameter specifies that a linear kernel should be used, which is suitable for linear classification tasks.
   - The `random_state=42` parameter ensures reproducibility of the results.

2. **Train the Model**:
   - The model is trained on the scaled training data (`X_train_scaled`) and the corresponding labels (`y_train`).

SVM is a powerful classification algorithm that works well for both linear and non-linear data (using different kernels). Standardizing the features ensures that the model performs optimally.

In [38]:
# Initialize the SVM model
svm_model = SVC(kernel="linear", random_state=42) 

# Train the model on the training set
svm_model.fit(X_train_scaled, y_train)

print("SVM model trained successfully!")


### Make Predictions Using the SVM Model

In this step, we use the trained Support Vector Machine (SVM) model to make predictions on the scaled test data.

1. **Generate Predictions**:
   - The `predict` method is used to generate predictions (`y_pred_svm`) for the test data (`X_test_scaled`).

These predictions will be used to evaluate the performance of the SVM model by comparing them to the actual labels (`y_test`).

In [39]:
# Predict on test data
y_pred_svm = svm_model.predict(X_test_scaled)

print("Predictions made successfully!")


### Evaluate the SVM Model

In this step, we evaluate the performance of the Support Vector Machine (SVM) model using various evaluation metrics.

1. **Compute Evaluation Metrics**:
   - **Accuracy**: Measures the proportion of correctly classified instances.
   - **Precision**: Measures the proportion of true positive predictions among all positive predictions (weighted average is used for multi-class classification).
   - **Recall**: Measures the proportion of true positives identified correctly (weighted average is used for multi-class classification).
   - **F1-Score**: The harmonic mean of precision and recall, providing a balance between the two (weighted average is used for multi-class classification).

2. **Print the Classification Report**:
   - The `classification_report` function provides a detailed breakdown of precision, recall, F1-score, and support for each class.

3. **Print Evaluation Scores**:
   - The accuracy, precision, recall, and F1-score are printed to summarize the model's performance.

These metrics help us understand how well the SVM model is performing on the test data and identify areas for improvement.

In [40]:
# Compute evaluation metrics
accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm, average="weighted")
recall_svm = recall_score(y_test, y_pred_svm, average="weighted")
f1_svm = f1_score(y_test, y_pred_svm, average="weighted")

# Print the classification report
print("📊 SVM Classification Report:\n")
print(classification_report(y_test, y_pred_svm))

# Print evaluation scores
print(f"Accuracy: {accuracy_svm:.4f}")
print(f"Precision: {precision_svm:.4f}")
print(f"Recall: {recall_svm:.4f}")
print(f"F1-Score: {f1_svm:.4f}")


***

### Compare Model Performance

In this step, we compare the performance of the three models (Logistic Regression, Random Forest, and SVM) by creating a summary DataFrame of their evaluation metrics.

1. **Create a Results DataFrame**:
   - A DataFrame is created to store the evaluation metrics (accuracy, precision, recall, and F1-score) for each model.
   - The models included are:
     - Logistic Regression
     - Random Forest
     - Support Vector Machine (SVM)

2. **Display the Results**:
   - The DataFrame is printed to provide a clear comparison of the models' performance.

This comparison helps us identify which model performs best on the test data based on the selected evaluation metrics.

In [41]:
# Create a DataFrame to compare results
results = pd.DataFrame({
    "Model": ["Logistic Regression", "Random Forest", "SVM"],
    "Accuracy": [accuracy_lr, accuracy_rf, accuracy_svm],
    "Precision": [precision_lr, precision_rf, precision_svm],
    "Recall": [recall_lr, recall_rf, recall_svm],
    "F1-Score": [f1_lr, f1_rf, f1_svm]
})

# Display results
print("Model Comparison:")
print(results)


In [44]:
# Collect results into a DataFrame
results = pd.DataFrame({
    "Model": ["Logistic Regression", "Random Forest", "SVM"],
    "Accuracy": [accuracy_lr, accuracy_rf, accuracy_svm],
    "Precision": [precision_lr, precision_rf, precision_svm],
    "Recall": [recall_lr, recall_rf, recall_svm],
    "F1-Score": [f1_lr, f1_rf, f1_svm]
})

# Plot results
plt.figure(figsize=(10,6))
results.set_index("Model").plot(kind="bar", figsize=(10,6), colormap="coolwarm")
plt.title("Classifier Performance Comparison")
plt.ylabel("Score")
plt.xticks(rotation=45)
plt.legend(loc="lower right")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()


### Summary of Model Performance

The table below compares the performance of three models (Logistic Regression, Random Forest, and SVM) based on accuracy, precision, recall, and F1-score:

| Model               | Accuracy  | Precision  | Recall     | F1-Score  |
|---------------------|-----------|------------|------------|-----------|
| Logistic Regression | 0.984466  | 0.984492   | 0.984466   | 0.984468  |
| Random Forest       | 0.981553  | 0.981606   | 0.981553   | 0.981548  |
| SVM                 | 0.986408  | 0.986424   | 0.986408   | 0.986409  |

#### Key Observations:
1. **SVM** performs the best across all metrics, achieving the highest accuracy, precision, recall, and F1-score.
2. **Logistic Regression** follows closely, with slightly lower scores compared to SVM.
3. **Random Forest** has the lowest scores among the three models but still performs very well, with all metrics above 0.98.

These results suggest that SVM is the most effective model for this dataset, but all three models demonstrate strong performance.